In [15]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm
from shutil import copyfile

%matplotlib inline

In [6]:
camera_type = 'sony'

settings = {}
if camera_type == 'iphone13':
    settings = {
        'ext': '.TIF',
        'splitter': 'IMG_',
        'need_crop': True,
        'crop_box': [800, 800, 3000, 2200],
        'folder': 'iphone13'
    }
elif camera_type == 'sony':
    settings = {
        'ext': '.tiff',
        'splitter': 'DSC',
        'need_crop': False,
        'folder': 'sony_a7c'
    }
    

images_folder = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/temp_checking/{}/raw/'.format(
    settings['folder']
)
images_folder = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/sonya7c/scene_2/raw/'

images_paths = [
    os.path.join(images_folder, fname) 
    for fname in os.listdir(images_folder)
    if fname.endswith(settings['ext'])
]
    

images_paths.sort(key=lambda x: int(os.path.splitext(x)[0].split(settings['splitter'])[-1]))
images_paths = images_paths[:10]

if settings['need_crop']:
    print('Use crop')
    x1, y1, x2, y2 = settings['crop_box']
    
    images = [
        cv2.cvtColor(
            cv2.imread(img_path, cv2.IMREAD_COLOR)[y1:y2, x1:x2],
            cv2.COLOR_BGR2RGB
        )
        for img_path in tqdm(images_paths)
    ]
else:
    images = [
        cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
        for img_path in tqdm(images_paths)
    ]
    
images = np.array(images)

  0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
base_path = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/sonya7c/'
gts_path = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/aligned_frames/'

os.makedirs(os.path.join(gts_path, 'clear/'), exists_ok=True)
os.makedirs(os.path.join(gts_path, 'noisy/'), exists_ok=True)

for scene in tqdm(os.listdir(base_path)):
    images_folder = os.path.join(base_path, scene, 'raw')
    images_paths = [
        os.path.join(images_folder, fname) 
        for fname in os.listdir(images_folder)
        if fname.endswith(settings['ext'])
    ]
    
    images_paths.sort(key=lambda x: int(os.path.splitext(x)[0].split(settings['splitter'])[-1]))
    clear_images_paths = images_paths[:10]
    noisy_images_paths = images_paths[10:]
    
    if settings['need_crop']:
        print('Use crop')
        x1, y1, x2, y2 = settings['crop_box']

        clear_images_paths = [
            cv2.cvtColor(
                cv2.imread(img_path, cv2.IMREAD_COLOR)[y1:y2, x1:x2],
                cv2.COLOR_BGR2RGB
            )
            for img_path in clear_images_paths
        ]
    else:
        clear_images = [
            cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
            for img_path in clear_images_paths
        ]

    clear_images = np.array(clear_images)
    clear_frame = clear_images.astype(np.float32).mean(axis=0).astype(np.uint8)
    
    for i, noisy_img_path in enumerate(noisy_images_paths):
        gt_path = os.path.join(gts_path, 'clear/', 'custom_{}_{}_{}.tiff'.format(camera_type, scene, i + 1))
        noisy_path = os.path.join(gts_path, 'noisy/', 'custom_{}_{}_{}.tiff'.format(camera_type, scene, i + 1))
        
        assert cv2.imwrite(gt_path, cv2.cvtColor(clear_frame, cv2.COLOR_RGB2BGR)), gt_path
        
#         if os.path.splitext(noisy_img_path)[1] == '.tiff':
#             copyfile(noisy_img_path, noisy_path)
#         else:
        _img = cv2.imread(noisy_img_path, cv2.IMREAD_COLOR)
        assert cv2.imwrite(noisy_path, _img), noisy_path

  0%|          | 0/8 [00:00<?, ?it/s]